In [16]:
import os
import gc
import csv
import argparse
import pickle

import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

from tqdm import tqdm

from dataset import ICUSepsisDataset
from model import SepsisPredictionModel

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SepsisPredictionModel(input_size=len(ICUSepsisDataset.features))
model.to(device)
model.load_state_dict(torch.load('model.state', map_location=device))

icu_test = ICUSepsisDataset('./data/test')

Found 10000 files in ./data/test


In [39]:
for x,y in icu_test:
    # ignore invalid samples
    if x is None or y is None:
        continue
    x = x.to(device).unsqueeze(0)
    y = y.to(device)  # squeeze since batch_size=1
    N = len(y)
    break

x.shape

torch.Size([1, 46, 9])

In [18]:
total = 0
correct = 0

with tqdm(total=len(icu_test)) as pbar:
    for x,y in icu_test:
        pbar.update(1)
        # ignore invalid samples
        if x is None or y is None:
            continue
        x = x.to(device).unsqueeze(0)
        y = y.to(device)  # squeeze since batch_size=1
        N = len(y)

        # Forward pass
        output = model(x).view(N,2)
        prediction = torch.softmax(output, dim=1).argmax(dim=1)

        assert prediction.shape == y.shape

        total += 1
        correct += int(prediction[-1] == y[-1])


100%|██████████| 10000/10000 [02:20<00:00, 71.34it/s]


In [20]:
total, correct, correct / total

(309929, 309361, 0.9981673221931474)

In [36]:
for x,y in icu_train:
    # ignore invalid samples
    if x is None or y is None:
        continue

    optimizer.zero_grad()

    x = x.to(device).unsqueeze(0)
    y = y.to(device)  # squeeze since batch_size=1
    N = len(y)

    # Forward pass
    output = model(x).view(N,2) # squeeze since batch_size=1
    L = loss(output, y)


KeyboardInterrupt: 

In [28]:
x.shape, y.shape, output.shape

(torch.Size([1, 23, 9]), torch.Size([1, 23]), torch.Size([1, 23, 2]))